In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import *

### api 불러오기

In [ ]:
!pip install spotipy

ERROR: Operation cancelled by user


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id='fdffc2b74db345b8b80f610ac64aaa92', client_secret='c643f4b185f14d768d25c719c98b9741')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, retries=50)

In [ ]:
import pandas as pd
PATH = '/content/drive/MyDrive/Colab 실습/논문'
liked_0 = pd.read_json(PATH + '/playlist/liked_0.json')['items']
liked_1 = pd.read_json(PATH + '/playlist/liked_1.json')['items']
liked_2 = pd.read_json(PATH + '/playlist/liked_2.json')['items']

In [ ]:
play = []
for i in range(len(liked_0)):
    play_dict = {}
    play_dict['artist_name'] = liked_0[i]['track']['artists'][0]['name']
    play_dict['track_name'] = liked_0[i]['track']['name']
    play_dict['track_id'] = liked_0[i]['track']['id']
    play_dict['popularity'] = liked_0[i]['track']['popularity']
    play_dict['liked'] = '0'
    play.append([play_dict])
for i in range(len(liked_1)):
    play_dict = {}
    play_dict['artist_name'] = liked_1[i]['track']['artists'][0]['name']
    play_dict['track_name'] = liked_1[i]['track']['name']
    play_dict['track_id'] = liked_1[i]['track']['id']
    play_dict['popularity'] = liked_1[i]['track']['popularity']
    play_dict['liked'] = '1'
    play.append([play_dict])
for i in range(len(liked_2)):
    play_dict = {}
    play_dict['artist_name'] = liked_2[i]['track']['artists'][0]['name']
    play_dict['track_name'] = liked_2[i]['track']['name']
    play_dict['track_id'] = liked_2[i]['track']['id']
    play_dict['popularity'] = liked_2[i]['track']['popularity']
    play_dict['liked'] = '1'
    play.append([play_dict])

In [ ]:
playlist = pd.DataFrame(columns = ['artist_name','track_name','liked','track_id','popularity'])
for item in play:
    playlist = playlist.append(item[0], ignore_index=True)
playlist

,artist_name,track_name,liked,track_id,popularity
0,호미들,사이렌,0,4eFXETUyAguBzHZ1KkOAAZ,44
1,호미들,넌 내게 안될거란 말을 했지만(prod. Kidstone),0,2ZJHVum4M1qRQ05I8IiF6B,33
2,호미들,무덤 위에 소주를 부어줘,0,5q6MTurViLpNl0fgHd76BA,39
3,창모,METEOR,0,4g6XOg9rvB55GCTJcYchOG,50
4,창모,아름다워,0,3fZWliK6TSGwx2feMCnu2Z,47
...,...,...,...,...,...
145,STAYC,YOUNG LUV,1,7HGKvoju3ucB7UqVt1GoJu,60
146,STAYC,SO BAD,1,13KR2yNRDrbKwR3dOKCDl4,63
147,The Volunteers,Summer,1,3aeZV1y0ck9CMIUIdeOjBj,45
148,5 Seconds of Summer,Youngblood,1,2iUXsYOEPhVqEBwsqP70rE,84


In [ ]:
track_features=[]
for i in range(len(liked_0)):
    track_features.append(sp.audio_features(liked_0[i]['track']['id']))
for i in range(len(liked_1)):
    track_features.append(sp.audio_features(liked_1[i]['track']['id']))
for i in range(len(liked_2)):
    track_features.append(sp.audio_features(liked_2[i]['track']['id']))

In [ ]:
tf_df = pd.DataFrame(columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'url', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])
for item in track_features:
    tf_df = tf_df.append(item[0], ignore_index=True)

In [ ]:
drop_cols = ['type','id','url','track_href','analysis_url','url','uri']
tf_df.drop(drop_cols,axis=1,inplace=True)

In [ ]:
playlist = pd.concat([playlist,tf_df],axis=1)

In [ ]:
playlist.keys()

In [ ]:
drop_cols = ['track_id']
playlist.drop(drop_cols,axis=1,inplace=True)

In [ ]:
playlist['lyrics'] = playlist['artist_name'].copy()
for i in range(len(playlist)):
    with open(PATH + f'/lyrics/{i}.txt', "r") as f:
        lyrics = f.read()
        playlist['lyrics'][i] = lyrics

### 형태소 tokenize

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
import re

def tokenize(text):
    word_tokens = re.findall(r"[a-zA-Z0-9]+", text) + re.findall(r"[가-힣]+", text)
    stop_words = stopwords.words('english')
    result = []
    for word in word_tokens:
        if word.lower() not in stop_words:
            result.append(word)
    return result

In [ ]:
playlist['token'] = playlist['lyrics'].apply(lambda x : tokenize(x))

In [ ]:
playlist

,Unnamed: 0,artist_name,track_name,liked,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics,token
0,0,호미들,사이렌,0,44,0.770,0.536,4,-8.052,1,0.0804,0.8650,0.000000,0.1030,0.642,140.015,200143,4,울려댔어 사이렌\n텅 빈 길거리엔\n도망치다 흘린 칼자루와 피가 흥건해\n우리 그때...,"[man, way, bro, payback, make, know, pain, saf..."
1,1,호미들,넌 내게 안될거란 말을 했지만(prod. Kidstone),0,33,0.681,0.585,8,-7.966,1,0.3210,0.6250,0.000001,0.1800,0.559,77.494,175102,4,넌 내게 안될꺼란 말을 했지만\n이제 나를 봐 이건 명품이야\n쌈박질만 했지만\n난...,"[oh, came, nothing, retire, 21, like, kg, cara..."
2,2,호미들,무덤 위에 소주를 부어줘,0,39,0.667,0.850,0,-2.648,0,0.0313,0.0951,0.000000,0.1570,0.290,145.040,191413,4,"난 맨몸으로 시작되어\n천 번은 슬프게 울음\n난 이제 새 신, 새 옷\n백 번은 ...","[tell, illusion, dont, like, bad, b, h, oh, ri..."
3,3,창모,METEOR,0,50,0.728,0.642,8,-4.989,1,0.0727,0.3160,0.000000,0.0834,0.755,185.795,197097,4,"Yeah, ha\nYeah, ha\nYeah, ha\nYeah, ha\nYeah, ...","[Yeah, ha, Yeah, ha, Yeah, ha, Yeah, ha, Yeah,..."
4,4,창모,아름다워,0,47,0.737,0.668,9,-5.675,1,0.0423,0.0390,0.000000,0.0800,0.344,87.933,220291,4,널 이제 놓아줘야 될것같애 의미없는 날짜를 셀것같애 내 손이 아닌 꽃이 무대인 나비...,"[bye, bay, diamond, luxury, thing, Run, Run, c..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,145,STAYC,YOUNG LUV,1,60,0.695,0.749,7,-4.187,0,0.0277,0.3920,0.000052,0.1040,0.458,129.905,206080,4,My love's so young\nI'm young 달콤한 나이\n너만 바라볼 순...,"[love, young, young, na, na, na, na, Cold, col..."
146,146,STAYC,SO BAD,1,63,0.557,0.906,7,-3.109,0,0.1030,0.1900,0.000000,0.0616,0.616,168.978,212265,4,"(STAYC)\nSTAYC girls, it's going down\n\nYou s...","[STAYC, STAYC, girls, going, stole, heart, hea..."
147,147,The Volunteers,Summer,1,45,0.424,0.633,6,-9.235,1,0.0283,0.0807,0.004850,0.2010,0.177,96.887,251033,4,Our eyes are closed\nBut we know when the sun ...,"[eyes, closed, know, sun, near, us, love, fore..."
148,148,5 Seconds of Summer,Youngblood,1,84,0.596,0.854,7,-5.114,0,0.4630,0.0169,0.000000,0.1240,0.152,120.274,203418,4,"Remember the words you told me, love me 'til t...","[Remember, words, told, love, til, day, die, S..."


In [ ]:
playlist.to_csv('playlist_processing.csv')